In [1]:
from query.models import FaceCharacterActor, Shot
from rekall.video_interval_collection import VideoIntervalCollection
from rekall.parsers import in_array, bbox_payload_parser, merge_dict_parsers, dict_payload_parser
from rekall.merge_ops import payload_plus
from rekall.payload_predicates import payload_satisfies
from rekall.spatial_predicates import scene_graph
from esper.rekall import intrvllists_to_result_bbox
from query.models import Face
from rekall.video_interval_collection import VideoIntervalCollection
from rekall.parsers import in_array, bbox_payload_parser
from rekall.merge_ops import payload_plus, merge_named_payload, payload_second
from esper.rekall import intrvllists_to_result_bbox
from rekall.payload_predicates import payload_satisfies
from rekall.list_predicates import length_at_most
from rekall.logical_predicates import and_pred, or_pred, true_pred
from rekall.spatial_predicates import scene_graph, make_region
from rekall.temporal_predicates import before, after, overlaps
from rekall.bbox_predicates import height_at_least
from esper.rekall import intrvllists_to_result, intrvllists_to_result_with_objects, add_intrvllists_to_result
from esper.prelude import esper_widget
from rekall.interval_list import Interval, IntervalList

In [2]:
RIGHT_HALF_MIN_X = 0.45
LEFT_HALF_MAX_X = 0.55
MIN_FACE_HEIGHT = 0.4
MAX_FACES_ON_SCREEN = 2
# faces are sampled every 12 frames
SAMPLING_RATE = 12
ONE_SECOND = 1
FOUR_SECONDS = 96
TEN_SECONDS = 240

In [3]:
# Annotate face rows with start and end frames and the video ID
faces_with_character_actor_qs = FaceCharacterActor.objects.annotate(
    min_frame=F('face__frame__number'),
    max_frame=F('face__frame__number'),
    video_id=F('face__frame__video_id'),
    bbox_x1=F('face__bbox_x1'),
    bbox_y1=F('face__bbox_y1'),
    bbox_x2=F('face__bbox_x2'),
    bbox_y2=F('face__bbox_y2'),
    character_name=F('characteractor__character__name')
)

In [4]:
faces_with_identity = VideoIntervalCollection.from_django_qs(
    faces_with_character_actor_qs,
    with_payload=in_array(merge_dict_parsers([
        bbox_payload_parser(VideoIntervalCollection.django_accessor),
        dict_payload_parser(VideoIntervalCollection.django_accessor, { 'character': 'character_name' }),
    ]))
).coalesce(payload_merge_op=payload_plus)

In [5]:
shots_qs = Shot.objects.filter(
    labeler=Labeler.objects.get(name='shot-hsvhist-face'))
shots = VideoIntervalCollection.from_django_qs(shots_qs)

In [6]:
def payload_unique_characters(payload1, payload2):
    if 'characters' not in payload1[0]:
        unique_characters = set([p['character'] for p in payload1])
        for p in payload2:
            unique_characters.add(p['character'])
        payload1[0]['characters'] = list(unique_characters)
    else:
        unique_characters = set([p['character'] for p in payload2])
        unique_characters.update(payload1[0]['characters'])
        payload1[0]['characters'] = list(unique_characters)
    return payload1
    
    

In [7]:
shots_with_faces = shots.merge(faces_with_identity, 
                               predicate=overlaps(), 
                               payload_merge_op=payload_second)

In [8]:
shots_with_faces = shots_with_faces.coalesce(payload_merge_op=payload_unique_characters)


In [9]:
def cross_product_faces(intrvl1, intrvl2):
    payload1 = intrvl1.get_payload()
    payload2 = intrvl2.get_payload()
    chrtrs1 = payload1[0]['characters'] if 'characters' in payload1[0] else list(set([p['character'] for p in payload1]))
    chrtrs2 = payload2[0]['characters'] if 'characters' in payload2[0] else list(set([p['character'] for p in payload2]))
    new_intervals = []
    for i in payload1:
        for j in chrtrs2:
            if i!=j:
                new_payload = {'A': i, 'B': j}
#                 new_payload.update()
                start = min(intrvl1.start, intrvl2.start)
                end = max(intrvl1.end, intrvl2.end)
#                 print(intrvl1.keys())
#                 print(intrvl1.video_id == intrvl2.video_id )
                new_intervals.append(Interval(start, end, {'A': i, 'B': j}))

    return new_intervals

In [10]:
def faces_equal(payload1, payload2):
    return (payload1['A'] == payload2['A'] and payload1['B'] == payload2['B']) or (payload1['A'] == payload2['B'] and payload1['B'] == payload2['A'])

In [ ]:
def faces_equal(payload1, payload2):
    if type(payload1) is not list and type(payload1) is not list:
        return (payload1['A'] == payload2['A'] and payload1['B'] == payload2['B']) or (payload1['A'] == payload2['B'] and payload1['B'] == payload2['A'])
    elif type(payload1) is list and type(payload1) is list:
        for i in payload1:
            for j in payload2:
                if i['A'] == j['A'] and i['B'] == j['B']:
                    return True
    elif type(payload1) is list:
        for i in payload1:
            if i['A'] == payload2['A'] and i['B'] == payload2['B']:
                return True
    else:
        for i in payload2:
            if i['A'] == payload1['A'] and i['B'] == payload1['B']:
                return True
    return False

def times_equal(intrvl1, intrvl2):
    return intrvl.start == intervl2.start and intrvl.end == intervl2.end

def merge_to_list(payload1, payload2):
    p1 = payload1 if type(payload1) is list else [payload1]
    p2 = payload2 if type(payload2) is list else [payload2]
    return p1+p2

In [11]:
two_shots = shots_with_faces.join(shots_with_faces, predicate=after(max_dist=ONE_SECOND, min_dist=ONE_SECOND), 
                                merge_op=cross_product_faces)


In [12]:
num_intervals = 0
for video_id in two_shots.intervals.keys():
    intvllist = two_shots.get_intervallist(video_id)
    s = intvllist.size()
    print(s)
    num_intervals += s
print(num_intervals)

401
48
2150
931
763
134
424
356
1191
706
965
627
743
874
537
552
483
1212
262
385
507
1035
679
260
903
1412
672
1091
739
464
1208
659
794
922
785
366
802
624
570
608
938
1911
1061
1493
937
185
1093
70
651
684
1388
950
1225
841
1012
439
506
506
1704
1168
972
1547
874
602
1336
1632
528
602
1564
1185
707
1049
1527
485
612
62828


In [13]:
conversations = two_shots.coalesce(predicate=payload_satisfies(faces_equal, arity=2))

In [14]:
num_intervals = 0
for video_id in conversations.intervals.keys():
    intvllist = conversations.get_intervallist(video_id)
    s = intvllist.size()
    print(s)
    num_intervals += s
print(num_intervals)

401
48
2150
931
763
134
424
1191
706
965
627
743
874
537
552
483
385
262
1212
507
1035
679
260
903
1336
1412
672
1091
739
464
1208
659
794
922
1547
802
841
624
570
608
890
1911
1061
1493
937
185
1093
70
684
1388
950
1225
785
1012
972
506
528
707
1632
1168
506
366
874
602
439
356
651
602
1564
1185
1704
1049
1527
485
612
62780


In [75]:
scene = three_shot.merge(three_shot, predicate=and_pred(after(max_dist=ONE_SECOND, min_dist=ONE_SECOND), 
                                              payload_satisfies(check_B_intersects, arity=2), arity=2)).coalesce()#, payload_merge_op=updateA))

In [15]:
esper_widget(intrvllists_to_result_with_objects(
            conversations.get_allintervals(), lambda payload, video: []),
            crop_bboxes=False,
            disable_playback=False,
            jupyter_keybindings=False)

VGridWidget(jsglobals={'queries': [['All faces', 'def all_faces():\n    from query.models import Face\n    fro…

In [ ]:
conversations.get_allintervals().key()

In [ ]:
conversations.intrvls.keys()

In [ ]:
conversations

In [101]:
def conversationsq(video_name):
    from query.models import FaceCharacterActor, Shot
    from rekall.video_interval_collection import VideoIntervalCollection
    from rekall.parsers import in_array, bbox_payload_parser, merge_dict_parsers, dict_payload_parser
    from rekall.merge_ops import payload_plus
    from rekall.payload_predicates import payload_satisfies
    from rekall.spatial_predicates import scene_graph
    from esper.rekall import intrvllists_to_result_bbox
    from query.models import Face
    from rekall.video_interval_collection import VideoIntervalCollection
    from rekall.parsers import in_array, bbox_payload_parser
    from rekall.merge_ops import payload_plus, merge_named_payload, payload_second
    from esper.rekall import intrvllists_to_result_bbox
    from rekall.payload_predicates import payload_satisfies
    from rekall.list_predicates import length_at_most
    from rekall.logical_predicates import and_pred, or_pred, true_pred
    from rekall.spatial_predicates import scene_graph, make_region
    from rekall.temporal_predicates import before, after, overlaps
    from rekall.bbox_predicates import height_at_least
    from esper.rekall import intrvllists_to_result, intrvllists_to_result_with_objects, add_intrvllists_to_result
    from esper.prelude import esper_widget
    from rekall.interval_list import Interval, IntervalList
    
    # faces are sampled every 12 frames
    SAMPLING_RATE = 12
    ONE_FRAME = 1
   

    faces_with_character_actor_qs = FaceCharacterActor.objects.annotate(
        min_frame=F('face__frame__number'),
        max_frame=F('face__frame__number'),
        video_id=F('face__frame__video_id'),
        bbox_x1=F('face__bbox_x1'),
        bbox_y1=F('face__bbox_y1'),
        bbox_x2=F('face__bbox_x2'),
        bbox_y2=F('face__bbox_y2'),
        character_name=F('characteractor__character__name')
    ).filter(face__frame__video__name__contains=video_name)
    
    faces_with_identity = VideoIntervalCollection.from_django_qs(
        faces_with_character_actor_qs,
        with_payload=in_array(merge_dict_parsers([
            bbox_payload_parser(VideoIntervalCollection.django_accessor),
            dict_payload_parser(VideoIntervalCollection.django_accessor, { 'character': 'character_name' }),
        ]))
    ).coalesce(payload_merge_op=payload_plus)

    shots_qs = Shot.objects.filter(
        labeler=Labeler.objects.get(name='shot-hsvhist-face'))
    shots = VideoIntervalCollection.from_django_qs(shots_qs)
    
    def payload_unique_characters(payload1, payload2):
        if 'characters' not in payload1[0]:
            unique_characters = set([p['character'] for p in payload1])
            for p in payload2:
                unique_characters.add(p['character'])
            payload1[0]['characters'] = list(unique_characters)
        else:
            unique_characters = set([p['character'] for p in payload2])
            unique_characters.update(payload1[0]['characters'])
            payload1[0]['characters'] = list(unique_characters)
        return payload1
        
    shots_with_faces = shots.merge(faces_with_identity, 
                                  predicate=overlaps(), 
                                  payload_merge_op=payload_second)
                                   
    shots_with_faces = shots_with_faces.coalesce(payload_merge_op=payload_unique_characters)

    def cross_product_faces(intrvl1, intrvl2):
        payload1 = intrvl1.get_payload()
        payload2 = intrvl2.get_payload()
        chrtrs1 = payload1[0]['characters'] if 'characters' in payload1[0] else list(set([p['character'] for p in payload1]))
        chrtrs2 = payload2[0]['characters'] if 'characters' in payload2[0] else list(set([p['character'] for p in payload2]))
        new_intervals = []
        for i in chrtrs1:
            for j in chrtrs2:
                if i!=j:
                    new_payload = {'A': i, 'B': j}
                    start = min(intrvl1.start, intrvl2.start)
                    end = max(intrvl1.end, intrvl2.end)
                    new_intervals.append(Interval(start, end, {'A': i, 'B': j}))

        return new_intervals
        
    def faces_equal(payload1, payload2):
        p1 = [payload1]
        if type(payload1) is dict and 'chrs' in payload1:
            p1 = payload1['chrs']
        elif type(payload1) is list:
            p1 = payload1
        
        p2 = [payload2]
        if type(payload2) is dict and 'chrs' in payload2:
            p2 = payload2['chrs']
        elif type(payload2) is list:
            p2 = payload2
            
        payload1 = p1
        payload2 = p2
        
        if type(payload1) is not list and type(payload1) is not list:
            return (payload1['A'] == payload2['A'] and payload1['B'] == payload2['B']) or (payload1['A'] == payload2['B'] and payload1['B'] == payload2['A'])
        elif type(payload1) is list and type(payload2) is list:
            for i in payload1:
                for j in payload2:
                    if i['A'] == j['A'] and i['B'] == j['B']:
                        return True
                    if i['A'] == j['B'] and i['B'] == j['A']:
                        return True
        elif type(payload1) is list:
            for i in payload1:
                if i['A'] == payload2['A'] and i['B'] == payload2['B']:
                    return True
                if i['A'] == payload2['B'] and i['B'] == payload2['A']:
                    return True
        else:
            for i in payload2:
                if i['A'] == payload1['A'] and i['B'] == payload1['B']:
                    return True
                if i['A'] == payload1['B'] and i['B'] == payload1['A']:
                    return True
        return False

    def times_equal(intrvl1, intrvl2):
        return (intrvl1.start >= intrvl2.start and intrvl1.end <= intrvl2.end) or (intrvl2.start >= intrvl1.start and intrvl2.end <= intrvl1.end)
        
    def times_overlap(intrvl1, intrvl2):
        return intrvl1.start <= intrvl2.end and intrvl2.start <= intrvl1.end
    
    def merge_to_list(payload1, payload2):
        p1 = payload1 if type(payload1) is list else [payload1]
        p2 = payload2 if type(payload2) is list else [payload2]
        return p1+p2
    
    def count_shots(payload1, payload2):
        p1 = [payload1]
        if type(payload1) is dict and 'chrs' in payload1:
            p1 = payload1['chrs']
        elif type(payload1) is list:
            p1 = payload1
        
        p2 = [payload2]
        if type(payload2) is dict and 'chrs' in payload2:
            p2 = payload2['chrs']
        elif type(payload2) is list:
            p2 = payload2
        
        p1_shots = payload1['shots'] if type(payload1) is dict and 'shots' in payload1 else 1
        p2_shots = payload2['shots'] if type(payload2) is dict and 'shots' in payload2 else 1
        return {'shots': p1_shots + p2_shots, 'chrs': p1 + p2}
        
    def shots_equal(payload1, payload2):
        p1 = [payload1]
        if type(payload1) is dict and 'chrs' in payload1:
            p1 = payload1['chrs']
        elif type(payload1) is list:
            p1 = payload1
        
        p2 = [payload2]
        if type(payload2) is dict and 'chrs' in payload2:
            p2 = payload2['chrs']
        elif type(payload2) is list:
            p2 = payload2
        
        p1_shots = payload1['shots'] if type(payload1) is dict and 'shots' in payload1 else 1
        p2_shots = payload2['shots'] if type(payload2) is dict and 'shots' in payload2 else 1

        shots = p1_shots if p1_shots > p2_shots else p2_shots
        return {'shots': shots, 'chrs': p1 + p2}

    two_shots = shots_with_faces.join(shots_with_faces, predicate=after(max_dist=ONE_FRAME, min_dist=ONE_FRAME), 
                                merge_op=cross_product_faces)

    convs = two_shots.coalesce(predicate=times_equal, payload_merge_op=merge_to_list)
    convs = convs.coalesce(predicate=payload_satisfies(faces_equal, arity=2), payload_merge_op=count_shots)
        
    adjacent_seq = convs.merge(convs, predicate=and_pred(after(max_dist=ONE_FRAME, min_dist=ONE_FRAME), payload_satisfies(faces_equal, arity=2), arity=2), payload_merge_op=count_shots)
    convs = convs.set_union(adjacent_seq)
    # convs = convs.coalesce(predicate=times_equal, payload_merge_op=shots_equal)

    def filter_fn(intvl):
        payload = intvl.get_payload()
        if type(payload) is dict and 'shots' in payload:
            return payload['shots'] >= 2
        return False 
    
    convs = convs.filter(filter_fn)
    convs = convs.coalesce(predicate=times_overlap)

    for video_id in convs.intervals.keys():
        print(video_id)
        intvllist = convs.get_intervallist(video_id)
        for intvl in intvllist.get_intervals():
            print(intvl.payload)
            print(str(intvl.start) + ':' + str(intvl.end))
    
    return convs

In [75]:
# Returns precision, recall, precision_per_item, recall_per_item
def compute_statistics(query_intrvllists, ground_truth_intrvllists):
    total_query_time = 0
    total_query_segments = 0
    total_ground_truth_time = 0
    total_ground_truth_segments = 0
    
    for video in query_intrvllists:
        total_query_time += query_intrvllists[video].coalesce().get_total_time()
        total_query_segments += query_intrvllists[video].size()
    for video in ground_truth_intrvllists:
        total_ground_truth_time += ground_truth_intrvllists[video].coalesce().get_total_time()
        total_ground_truth_segments += ground_truth_intrvllists[video].size()
        
    total_overlap_time = 0
    overlapping_query_segments = 0
    overlapping_ground_truth_segments = 0
    
    for video in query_intrvllists:
        if video in ground_truth_intrvllists:
            query_list = query_intrvllists[video]
            gt_list = ground_truth_intrvllists[video]
            
            total_overlap_time += query_list.overlaps(gt_list).coalesce().get_total_time()
            overlapping_query_segments += query_list.filter_against(gt_list, predicate=overlaps()).size()
            overlapping_ground_truth_segments += gt_list.filter_against(query_list, predicate=overlaps()).size()
    
    if total_query_time == 0:
        precision = 1.0
        precision_per_item = 1.0
    else:
        precision = total_overlap_time / total_query_time
        precision_per_item = overlapping_query_segments / total_query_segments
    
    if total_ground_truth_time == 0:
        recall = 1.0
        recall_per_item = 1.0
    else:
        recall = total_overlap_time / total_ground_truth_time
        recall_per_item = overlapping_ground_truth_segments / total_ground_truth_segments
    
    return precision, recall, precision_per_item, recall_per_item

def print_statistics(query_intrvllists, ground_truth_intrvllists):
    precision, recall, precision_per_item, recall_per_item = compute_statistics(
        query_intrvllists, ground_truth_intrvllists)

    print("Precision: ", precision)
    print("Recall: ", recall)
    print("Precision Per Item: ", precision_per_item)
    print("Recall Per Item: ", recall_per_item)

In [113]:
godfather_query = conversationsq('the godfather part iii')
for k in godfather_query.intervals.keys():
    print(k)
godfather_query = godfather_query.filter(lambda inv: inv.start < 43103)
godfather_query = {'216': godfather_query.get_intervallist(216)}

data = [(12481, 13454), (13673, 14729), (16888, 17299), (21101, 27196), (27602, 29032), (29033, 33204), (34071, 41293), (41512, 43103)]
godfather_gt = {'216': IntervalList([Interval(start, end, payload=None) for (start,end) in data])}
print_statistics(godfather_query, godfather_gt)


216
{'chrs': [{'B': 'mary corleone', 'A': 'don michael corleone'}, {'B': 'kay adams michelson', 'A': 'don michael corleone'}, {'B': 'anthony vito corleone', 'A': 'don michael corleone'}, {'B': 'don michael corleone', 'A': 'mary corleone'}, {'B': 'don michael corleone', 'A': 'anthony vito corleone'}, {'B': 'mary corleone', 'A': 'don michael corleone'}, {'B': 'anthony vito corleone', 'A': 'don michael corleone'}], 'shots': 3}
3241:4282
{'chrs': [{'B': 'don michael corleone', 'A': 'kay adams michelson'}, {'B': 'b.j. harrison', 'A': 'kay adams michelson'}, {'B': 'mary corleone', 'A': 'kay adams michelson'}, {'B': 'anthony vito corleone', 'A': 'kay adams michelson'}, {'B': 'kay adams michelson', 'A': 'don michael corleone'}, {'B': 'kay adams michelson', 'A': 'b.j. harrison'}, {'B': 'kay adams michelson', 'A': 'archbishop gilday'}, {'B': 'kay adams michelson', 'A': 'connie corleone rizzi'}], 'shots': 2}
7877:8347
{'chrs': [{'B': 'connie corleone rizzi', 'A': 'don michael corleone'}, {'B': 'c

Recall:  0.9028280099350734
Precision Per Item:  0.5555555555555556
Recall Per Item:  1.0


In [111]:
apollo_query = conversationsq('apollo 13')
apollo_query = apollo_query.filter(lambda inv: inv.start < 34618)
apollo_query = {'15': apollo_query.get_intervallist(15)}


15
{'chrs': [{'B': 'jim lovell', 'A': 'jack swigert'}, {'B': 'gene kranz', 'A': 'jack swigert'}, {'B': 'jack swigert', 'A': 'gene kranz'}, {'B': 'jim lovell', 'A': 'gene kranz'}, {'B': 'jack swigert', 'A': 'jim lovell'}, {'B': 'gene kranz', 'A': 'jim lovell'}, {'B': 'jack swigert', 'A': 'marilyn lovell'}, {'B': 'gene kranz', 'A': 'marilyn lovell'}], 'shots': 2}
2578:4100
{'chrs': [{'B': 'ken mattingly', 'A': 'fred haise'}, {'B': 'fred haise', 'A': 'ken mattingly'}, {'B': 'ken mattingly', 'A': 'fred haise'}], 'shots': 3}
4244:4826
{'chrs': [{'B': 'fred haise', 'A': 'jim lovell'}, {'B': 'marilyn lovell', 'A': 'jim lovell'}, {'B': 'gene kranz', 'A': 'jim lovell'}, {'B': 'ken mattingly', 'A': 'jim lovell'}, {'B': 'fred haise', 'A': 'marilyn lovell'}, {'B': 'jim lovell', 'A': 'marilyn lovell'}, {'B': 'gene kranz', 'A': 'marilyn lovell'}, {'B': 'ken mattingly', 'A': 'marilyn lovell'}, {'B': 'marilyn lovell', 'A': 'jim lovell'}, {'B': 'jim lovell', 'A': 'marilyn lovell'}, {'B': 'jim lovell', 

In [112]:
data = [(2578, 4100), (4244, 4826), (5098, 5828), (7757, 9546), (9602, 10300), (12393, 12943), (13088, 13884), (14146, 15212), (15427, 16116), (18040, 19198), (20801, 23368), (24572, 26185), (26735, 28753), (29462, 30873), (31768, 34618)]
apollo_gt = {'15': IntervalList([Interval(start, end, payload=None) for (start,end) in data])}
print_statistics(apollo_query, apollo_gt)

Precision:  0.9801451458304806
Recall:  0.7144069065322621
Precision Per Item:  1.0
Recall Per Item:  0.9333333333333333


In [50]:
invllist = caption_metadata_for_video(15)

In [109]:
hp2_query = conversationsq('harry potter and the chamber')
hp2_query = hp2_query.filter(lambda inv: inv.start < 20308)
hp2_query = {'374': hp2_query.get_intervallist(374)}

374
{'chrs': [{'B': 'harry potter', 'A': 'gilderoy lockhart'}, {'B': 'gilderoy lockhart', 'A': 'harry potter'}], 'shots': 2}
2409:2816
{'chrs': [{'B': 'harry potter', 'A': 'dobby'}, {'B': 'dobby', 'A': 'harry potter'}, {'B': 'harry potter', 'A': 'dobby'}, {'B': 'dobby', 'A': 'harry potter'}, {'B': 'harry potter', 'A': 'dobby'}, {'B': 'dobby', 'A': 'harry potter'}, {'B': 'harry potter', 'A': 'dobby'}, {'B': 'dobby', 'A': 'harry potter'}], 'shots': 8}
4752:5958
{'chrs': [{'B': 'dobby', 'A': 'harry potter'}, {'B': 'harry potter', 'A': 'dobby'}, {'B': 'dobby', 'A': 'harry potter'}, {'B': 'harry potter', 'A': 'dobby'}, {'B': 'dobby', 'A': 'harry potter'}, {'B': 'harry potter', 'A': 'dobby'}, {'B': 'dobby', 'A': 'harry potter'}, {'B': 'harry potter', 'A': 'dobby'}], 'shots': 8}
6440:7916
{'chrs': [{'B': 'dobby', 'A': 'harry potter'}, {'B': 'dobby', 'A': 'tom riddle'}, {'B': 'harry potter', 'A': 'tom riddle'}, {'B': 'tom riddle', 'A': 'harry potter'}], 'shots': 2}
8085:8503
{'chrs': [{'B': 'h

In [110]:
data = [(2155, 4338), (4687, 6188), (6440, 10134), (12921, 13151), (16795, 17370), (17766, 18021), (18102, 19495), (19622, 20308)]
hp2_gt = {'374': IntervalList([Interval(start, end, payload=None) for (start,end) in data])}
print_statistics(hp2_query, hp2_gt)

Precision:  0.8393842579146094
Recall:  0.5495863839497955
Precision Per Item:  0.75
Recall Per Item:  0.875


In [114]:
fc_query = conversationsq('fight club')
fc_query = fc_query.filter(lambda inv: inv.start < 58258)
fc_query = {'61': fc_query.get_intervallist(61)}

61
{'chrs': [{'B': "robert 'bob' paulsen", 'A': 'the narrator'}, {'B': 'the narrator', 'A': "robert 'bob' paulsen"}, {'B': "robert 'bob' paulsen", 'A': 'the narrator'}], 'shots': 3}
5085:5602
{'chrs': [{'B': 'the narrator', 'A': 'richard chesler'}, {'B': 'richard chesler', 'A': 'the narrator'}, {'B': 'the narrator', 'A': 'richard chesler'}, {'B': 'richard chesler', 'A': 'the narrator'}], 'shots': 4}
6493:6814
{'chrs': [{'B': "robert 'bob' paulsen", 'A': 'the narrator'}, {'B': 'the narrator', 'A': "robert 'bob' paulsen"}, {'B': 'the narrator', 'A': "robert 'bob' paulsen"}], 'shots': 3}
10299:10561
{'chrs': [{'B': 'the narrator', 'A': "robert 'bob' paulsen"}, {'B': "robert 'bob' paulsen", 'A': 'the narrator'}], 'shots': 2}
10731:13376
{'chrs': [{'B': 'the narrator', 'A': 'marla singer'}, {'B': "robert 'bob' paulsen", 'A': 'marla singer'}, {'B': 'marla singer', 'A': 'the narrator'}, {'B': 'the narrator', 'A': 'marla singer'}, {'B': 'marla singer', 'A': 'the narrator'}, {'B': 'marla singer

In [115]:
data = [(4698, 5602), (6493, 6865), (8670, 9156), (9517, 10908), (11087, 13538), (22039, 24188), (25603, 27656), (31844, 32812), (32918, 33451), (33698, 35363), (42072, 45143), (45272, 46685), (49162, 50618), (56830, 58258)]
fc_gt = {'61': IntervalList([Interval(start, end, payload=None) for (start,end) in data])}
print_statistics(fc_query, fc_gt)

Precision:  0.7107177395618719
Recall:  0.8310226155358899
Precision Per Item:  0.6666666666666666
Recall Per Item:  0.9285714285714286


In [51]:
for intvl in invllist.get_intervals():
    if 'speaker' in intvl.payload:
        print(intvl.payload)

{'man_end': 46.917, 'full_line': "The crew's crossing gantry for capsule ingress.", 'aligned': True, 'man_start': 44.744, 'speaker': "man's voice "}
{'man_end': 48.548, 'full_line': 'Roger that.', 'aligned': True, 'man_start': 47.013, 'speaker': "2nd man's voice "}
{'man_end': 51.625, 'full_line': 'Inspired by the late President Kennedy', 'aligned': True, 'man_start': 49.248, 'speaker': 'walter cronkite '}
{'man_end': 56.633, 'full_line': 'in only seven years, America has risen', 'aligned': True, 'man_start': 51.684, 'speaker': 'walter cronkite '}
{'man_end': 56.633, 'full_line': 'to the challenge of what he called,', 'aligned': True, 'man_start': 51.684, 'speaker': 'walter cronkite '}
{'man_end': 60.493, 'full_line': '"The most hazardous and dangerous', 'aligned': True, 'man_start': 56.989, 'speaker': 'walter cronkite '}
{'man_end': 60.493, 'full_line': 'and greatest adventure', 'aligned': True, 'man_start': 56.989, 'speaker': 'walter cronkite '}
{'man_end': 63.072, 'full_line': '"on 

{'man_end': 6018.378, 'full_line': 'The re-entry corridor is, in fact, so narrow', 'aligned': True, 'man_start': 6013.873, 'speaker': 'news anchor '}
{'man_end': 6018.378, 'full_line': 'that if this basketball were the Earth', 'aligned': True, 'man_start': 6013.873, 'speaker': 'news anchor '}
{'man_end': 6022.62, 'full_line': 'and this softball were the moon,', 'aligned': True, 'man_start': 6018.445, 'speaker': 'news anchor '}
{'man_end': 6022.62, 'full_line': 'and the two were placed 14 feet apart', 'aligned': True, 'man_start': 6018.445, 'speaker': 'news anchor '}
{'man_end': 6026.991, 'full_line': 'the crew would have to hit a target', 'aligned': True, 'man_start': 6022.816, 'speaker': 'news anchor '}
{'man_end': 6026.991, 'full_line': 'no thicker than this piece of paper.', 'aligned': True, 'man_start': 6022.816, 'speaker': 'news anchor '}
{'man_end': 6029.567, 'full_line': "On your toes. We're doing this one blind.", 'aligned': True, 'man_start': 6027.087, 'speaker': 'glynn lunney

In [ ]:
Apollo 13
1:48 --> 2:50 V
2:57 --> 3:20 V
5:24 --> 7:44 V (5:24 - 6:24) (6:45 - 7:18) (shot broken up because of shots of the moon)
8:26 --> 9:02; 8:36 - 9:02 
10:00 --> 10:33; 9:41 - 10:33 - long shot times w multiple people present
10:33 --> 11:11; 10:44 - 11:12 - skipped the daughter
12:40 --> 13:17; 12:33 - 13:21 (shot is a bit over extended)
                [14:28 - 14:51] - reaction sequence; not dialogue
17:03 -- 18:02;  18:15 ; V - over extended; catches him in the next scene
20:29 --> 21:27 [DID NOT CATCH]
22:04 --> 23:56 V
27:04 --> 27:30 [caught and 
27:34 --> 27:47 combined in unexpected ways
                 
Godfather

                 
                 
                 
data = [
    (8757,9049),
    (12750,13463),
    (13683,14227),
    (21357,22236),
    (22294,22758),
    (23147,25854),
    (26007,26942),
    (27620,28172),
    (28382,28623),
    (28785,29036),
    (29904,31014),
    (33936,35339),
    (35421,36248),
    (39388,40062),
    (41675,42689),
    (51246,52118),
    (53117,54776),
    (54895,55762),
    (56819,59963),
    (60253,61875),
    (66533,67846),
    (68729,69040),
    (69421,70153),
    (70285,71102)]
intrvllist = IntervalList([Interval(start, end, payload=None) for (start,end) in data])
shot_reverse_shot_labelled = {216: intrvllist}
esper_widget(intrvllists_to_result_with_objects(shot_reverse_shot_labelled, lambda payload, video: []), disable_captions=True)